# NSE control setup with pyMOR

In [1]:
from data_utils import (solve_steadystate_nse, get_stokes_solution, linearized_convection, 
                        writevp_paraview, collect_vtu_files, eva_quadterm)

import numpy as np
import os
import pickle
import scipy
import scipy.io as spio
import scipy.linalg as spla
import scipy.sparse as sps
import scipy.sparse.linalg as spsla

 ### Setup Parameters

In [2]:
Re = 110          # Reynolds number
control = 'bc'    # 'dist' for distributed control, 'bc' for control via boundary conditions or None
level = 2         # discretization level (1, 2 or 3)
palpha = 1e-3     # penalty for Robin-boundary (only in control == 'bc' case)

# some strings for data storage
data_str = 'data/' + 'lvl_' + str(level) + ('_' + control if control is not None else '')
setup_str = data_str + '/re_' + str(Re) + ('_palpha_' + str(palpha) if control=='bc' else '')

if not os.path.exists(setup_str):
    os.makedirs(setup_str)

In [3]:
mats = spio.loadmat(data_str + '/mats')

M = mats['M']
J = mats['J']
hmat = mats['H']
fv = mats['fv'] + 1./Re*mats['fv_diff'] + mats['fv_conv']
fp = mats['fp'] + mats['fp_div']
pcmat = mats['Cp']
vcmat = mats['Cv']
NV, NP = fv.shape[0], fp.shape[0]

if control == 'dist':
    A = 1./Re*mats['A'] + mats['L1'] + mats['L2']
    B = mats['B']
    # restrict to less dofs in the input
    NU = B.shape[1]
    B  = B[:, [0, NU//2]]
elif control == 'bc':
    A = 1./Re*mats['A'] + mats['L1'] + mats['L2'] + 1./palpha*mats['Arob']
    B = 1./palpha*mats['Brob']
else:
    A = 1./Re*mats['A'] + mats['L1'] + mats['L2']

### Steady-state NSE solution and linearized convection

In [4]:
if not os.path.isfile(setup_str + '/ss_nse_sol'):
    # compute steady-state nse solution as linearization point
    ss_nse_v, _ = solve_steadystate_nse(mats, Re, control, palpha=palpha)
    
    # compute linearized convection
    conv_mat = linearized_convection(mats['H'], ss_nse_v)

    scipy.io.savemat(setup_str + '/ss_nse_sol', {'ss_nse_v': ss_nse_v, 'conv_mat': conv_mat})

else:
    ss_nse_sol = scipy.io.loadmat(setup_str + '/ss_nse_sol')
    ss_nse_v, conv_mat = ss_nse_sol['ss_nse_v'], ss_nse_sol['conv_mat']

Iteration step 0 (Picard)
Norm of nse residual:   4.480292e-02
Norm of current update: 1.000000e+00


Iteration step 1 (Picard)
Norm of nse residual:   1.746994e-02
Norm of current update: 2.791143e-01


Iteration step 2 (Picard)
Norm of nse residual:   7.494149e-03
Norm of current update: 1.430542e-01


Iteration step 3 (Picard)
Norm of nse residual:   6.918089e-03
Norm of current update: 1.137928e-01


Iteration step 4 (Picard)
Norm of nse residual:   7.034428e-03
Norm of current update: 8.932566e-02


Iteration step 5 (Newton)
Norm of nse residual:   1.567080e-03
Norm of current update: 1.301440e-01


Iteration step 6 (Newton)
Norm of nse residual:   8.988354e-05
Norm of current update: 3.087307e-02


Iteration step 7 (Newton)
Norm of nse residual:   7.140975e-07
Norm of current update: 2.291298e-03


Iteration step 8 (Newton)
Norm of nse residual:   3.877240e-11
Norm of current update: 2.103306e-05


Iteration step 9 (Newton)
Norm of nse residual:   9.707244e-16
Norm of current upd

### Reduce linearization with pyMOR

In [5]:
from pymor.models.iosys import StokesDescriptorModel
from pymor.reductors.h2 import StokesGapIRKAReductor
from pymor.reductors.bt import StokesLQGBTReductor
from pymor.operators.numpy import NumpyMatrixOperator
from pymor.algorithms.to_matrix import to_matrix

# MOR setup
method = 'GapIRKA'
rom_ord = 30
tol = 1e-3

rom_str = setup_str + '/roms/' + method + '_' + str(rom_ord) + '_' + str(tol)

if not os.path.isfile(rom_str):
    if not os.path.exists(setup_str + '/roms'):
        os.makedirs(setup_str + '/roms')

    if control == 'dist':
        Aop = NumpyMatrixOperator(-1./Re * mats['A'] - conv_mat)
    elif control == 'bc':
        Aop = NumpyMatrixOperator(-1./Re * mats['A'] - 1./palpha*mats['Arob'] - conv_mat)

    Eop = NumpyMatrixOperator(M)
    Gop = NumpyMatrixOperator(J.T)
    Bop = NumpyMatrixOperator(B)
    Cop = NumpyMatrixOperator(vcmat)
    fom = StokesDescriptorModel(Aop, Gop, Bop, Cop, None, Eop)

    if method == 'GapIRKA':
        reductor = StokesGapIRKAReductor(fom)
        rom = reductor.reduce(rom_ord, tol=tol, conv_crit='sigma')
    else:
        reductor = StokesLQGBTReductor(fom)
        rom = reductor.reduce(rom_ord, tol=tol)
        
    with open(rom_str, 'wb') as rom_file:
        pickle.dump({'reductor': reductor, 'rom': rom}, rom_file)

else:
    with open(rom_str, 'rb') as rom_file: 
        rom_dict = pickle.load(rom_file)
    
    rom = rom_dict['rom']
    reductor = rom_dict['reductor']
    

00:00 StokesGapIRKAReductor: Generating initial interpolation data
00:00 StokesGapIRKAReductor: Starting gap IRKA
00:10 gram_schmidt: Orthonormalizing vector 2 again
00:10 gram_schmidt: Orthonormalizing vector 3 again
00:10 gram_schmidt: Orthonormalizing vector 4 again
00:10 gram_schmidt: Orthonormalizing vector 5 again
00:10 gram_schmidt: Orthonormalizing vector 6 again
00:10 gram_schmidt: Orthonormalizing vector 7 again
00:10 gram_schmidt: Orthonormalizing vector 8 again
00:10 gram_schmidt: Orthonormalizing vector 9 again
00:10 gram_schmidt: Orthonormalizing vector 10 again
00:10 gram_schmidt: Orthonormalizing vector 11 again
00:10 gram_schmidt: Orthonormalizing vector 12 again
00:10 gram_schmidt: Orthonormalizing vector 13 again
00:10 gram_schmidt: Orthonormalizing vector 14 again
00:10 gram_schmidt: Orthonormalizing vector 15 again
00:10 gram_schmidt: Orthonormalizing vector 16 again
00:10 gram_schmidt: Orthonormalizing vector 17 again
00:10 gram_schmidt: Orthonormalizing vector 18

02:16 gram_schmidt: Orthonormalizing vector 24 again
02:16 gram_schmidt: Orthonormalizing vector 25 again
02:16 gram_schmidt: Orthonormalizing vector 26 again
02:16 gram_schmidt: Orthonormalizing vector 27 again
02:16 gram_schmidt: Orthonormalizing vector 28 again
02:16 gram_schmidt: Orthonormalizing vector 29 again
02:26 gram_schmidt: Orthonormalizing vector 26 again
02:26 gram_schmidt: Orthonormalizing vector 28 again
02:26 LTIPGReductor: Operator projection ...
02:26 LTIPGReductor: Building ROM ...
02:26 StokesGapIRKAReductor: Convergence criterion in iteration 7: 2.110708e+00
02:35 gram_schmidt: Orthonormalizing vector 10 again
02:35 gram_schmidt: Orthonormalizing vector 16 again
02:35 gram_schmidt: Orthonormalizing vector 19 again
02:35 gram_schmidt: Orthonormalizing vector 20 again
02:35 gram_schmidt: Orthonormalizing vector 23 again
02:35 gram_schmidt: Orthonormalizing vector 24 again
02:35 gram_schmidt: Orthonormalizing vector 25 again
02:35 gram_schmidt: Orthonormalizing vecto

05:32 gram_schmidt: Orthonormalizing vector 9 again
05:32 gram_schmidt: Orthonormalizing vector 12 again
05:33 gram_schmidt: Orthonormalizing vector 18 again
05:33 gram_schmidt: Orthonormalizing vector 20 again
05:33 gram_schmidt: Orthonormalizing vector 23 again
05:33 gram_schmidt: Orthonormalizing vector 24 again
05:33 gram_schmidt: Orthonormalizing vector 26 again
05:33 gram_schmidt: Orthonormalizing vector 27 again
05:33 gram_schmidt: Orthonormalizing vector 28 again
05:33 gram_schmidt: Orthonormalizing vector 29 again
05:42 gram_schmidt: Orthonormalizing vector 23 again
05:42 LTIPGReductor: Operator projection ...
05:42 LTIPGReductor: Building ROM ...
05:42 StokesGapIRKAReductor: Convergence criterion in iteration 17: 4.147012e+00
05:52 gram_schmidt: Orthonormalizing vector 12 again
05:52 gram_schmidt: Orthonormalizing vector 18 again
05:52 gram_schmidt: Orthonormalizing vector 20 again
05:52 gram_schmidt: Orthonormalizing vector 23 again
05:52 gram_schmidt: Orthonormalizing vecto

09:27 gram_schmidt: Orthonormalizing vector 10 again
09:27 gram_schmidt: Orthonormalizing vector 12 again
09:27 gram_schmidt: Orthonormalizing vector 18 again
09:27 gram_schmidt: Orthonormalizing vector 20 again
09:27 gram_schmidt: Orthonormalizing vector 23 again
09:27 gram_schmidt: Orthonormalizing vector 24 again
09:27 gram_schmidt: Orthonormalizing vector 26 again
09:27 gram_schmidt: Orthonormalizing vector 28 again
09:27 gram_schmidt: Orthonormalizing vector 29 again
09:37 gram_schmidt: Orthonormalizing vector 23 again
09:37 LTIPGReductor: Operator projection ...
09:37 LTIPGReductor: Building ROM ...
09:37 StokesGapIRKAReductor: Convergence criterion in iteration 29: 1.297151e-03
09:47 gram_schmidt: Orthonormalizing vector 10 again
09:47 gram_schmidt: Orthonormalizing vector 12 again
09:47 gram_schmidt: Orthonormalizing vector 18 again
09:47 gram_schmidt: Orthonormalizing vector 20 again
09:47 gram_schmidt: Orthonormalizing vector 23 again
09:47 gram_schmidt: Orthonormalizing vect

### Stabilizing feedback

In [6]:
Arom = to_matrix(rom.A, format='dense')
Erom = to_matrix(rom.E, format='dense')
Brom = to_matrix(rom.B, format='dense')
Crom = to_matrix(rom.C, format='dense')

XCARE = spla.solve_continuous_are(Arom, Brom, Crom.T @ Crom, np.eye(Brom.shape[1]), e=Erom, balanced=False)

# stabilizing feedback for ROM
Krom = Brom.T @ XCARE @ Erom

# stabilizing feedback for FOM
K = reductor.reconstruct(rom.A.source.from_numpy(Krom)).to_numpy()

In [16]:
# compute Stokes solution as inital value for time stepping
stksv, stksp = get_stokes_solution(mats, Re, control, palpha=palpha, ct0=0)

if control is None:
    def bbcu(v):
        return np.zeros((NV, 1))
else:
    Kw = K @ stksv
    # define control based on stabilizing feedback
    def bbcu(v):
        uvec = -(K @ v - Kw)
        return B @ uvec

### Simulation with stabilization

In [17]:
# parameters for time-stepping
t0 = 0.
tE = 8.
Nts = 2**12
DT = (tE-t0)/Nts
trange = np.linspace(t0, tE, Nts+1)

# files for results and visualization
if not os.path.exists(setup_str + '/results'):
    os.makedirs(setup_str + '/results')

poutlist = []
voutlist = []
vfile = lambda t : setup_str + '/results/v_t{0}.vtu'.format(t)
pfile = lambda t : setup_str + '/results/p_t{0}.vtu'.format(t)
vfilerel = lambda t : 'results/v_t{0}.vtu'.format(t)
pfilerel = lambda t : 'results/p_t{0}.vtu'.format(t)
vfilelist = []
pfilelist = []
strtojson = data_str + '/visualization.jsn'

In [18]:
old_v = stksv

sysmat = sps.vstack([
             sps.hstack([M+DT*A, -J.T]), 
             sps.hstack([J, sps.csc_matrix((NP, NP))])
         ]).tocsc()
sysmati = spsla.factorized(sysmat)

for k, t in enumerate(trange):
    crhsv = M*old_v + DT*(fv - eva_quadterm(hmat, old_v) + bbcu(old_v))
    crhs = np.vstack([crhsv, fp])
    vp_new = np.atleast_2d(sysmati(crhs.flatten())).T
    old_v = vp_new[:NV]
    p = vp_new[NV:]

    poutlist.append((pcmat*p)[0][0])
    voutlist.append((vcmat*old_v).flatten())
    if np.mod(k, round(Nts/64)) == 0:
        print('timestep {0:4d}/{1}, t={2:f}, |v|={3:e}'.format(k, Nts, t, np.linalg.norm(old_v)))
        writevp_paraview(velvec=old_v, pvec=p, vfile=vfile(t), pfile=pfile(t), strtojson=strtojson)
        vfilelist.append(vfilerel(t))
        pfilelist.append(pfilerel(t))

timestep    0/4096, t=0.000000, |v|=5.492109e+01
timestep   64/4096, t=0.125000, |v|=5.566639e+01
timestep  128/4096, t=0.250000, |v|=5.619453e+01
timestep  192/4096, t=0.375000, |v|=5.636728e+01
timestep  256/4096, t=0.500000, |v|=5.639891e+01
timestep  320/4096, t=0.625000, |v|=5.639396e+01
timestep  384/4096, t=0.750000, |v|=5.633572e+01
timestep  448/4096, t=0.875000, |v|=5.623688e+01
timestep  512/4096, t=1.000000, |v|=5.601854e+01
timestep  576/4096, t=1.125000, |v|=5.595385e+01
timestep  640/4096, t=1.250000, |v|=5.606351e+01
timestep  704/4096, t=1.375000, |v|=5.605803e+01
timestep  768/4096, t=1.500000, |v|=5.598224e+01
timestep  832/4096, t=1.625000, |v|=5.591002e+01
timestep  896/4096, t=1.750000, |v|=5.623332e+01
timestep  960/4096, t=1.875000, |v|=5.632636e+01
timestep 1024/4096, t=2.000000, |v|=5.639765e+01
timestep 1088/4096, t=2.125000, |v|=5.624908e+01
timestep 1152/4096, t=2.250000, |v|=5.660051e+01
timestep 1216/4096, t=2.375000, |v|=5.667669e+01
timestep 1280/4096, 

### For visualization run 'paraview v_results.pvd'

In [19]:
collect_vtu_files(vfilelist, setup_str + '/v_results.pvd')
collect_vtu_files(pfilelist, setup_str + '/p_results.pvd')